In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
from scipy import stats, spatial
import pickle
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
import csv
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.colorbar import colorbar
from collections import Counter

/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:16: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  app.launch_new_instance()


In [3]:
tfidf_path = '../../data/tfidf_20230115_largest_rel/'

In [34]:
lda_df = pd.read_csv('../../data/' + 'harry_potter_lda_sample_with_text.tsv', sep='\t')

In [35]:
lda_df.head(2)

,Unnamed: 0,index,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,...,Title,Words,Text,PublishDate,UpdateDate,CompleteDate,Comments,URL,Dist,JSD
0,0,124,"Age Regression/De-Aging, Kidnapping, Memory Lo...",Creator Chose Not To Use Archive Warnings,Ischa,49,M/M,1,"Pansy Parkinson, Blaise Zabini",['Harry Potter - J. K. Rowling'],...,The Picket Fence,5566.0,\n\n~One~\n\n\xe2\x80\x9cYou\xe2\x80\x99ve got...,2013-10-03,NaN,2013-10-03,36,http://archiveofourown.org/works/989371?view_a...,"[3.5611445e-06, 1.0047966e-05, 3.5611445e-06, ...",0.280998
1,1,539,"Angst, Dementors (Harry Potter) - Freeform, Dark",Creator Chose Not To Use Archive Warnings,Frayach,7,M/M,1,"Harry Potter, Draco Malfoy, Hermione Granger",['Harry Potter - J. K. Rowling'],...,The Kiss,5175.0,"\nNot just a kiss, but The Kiss. The kind of k...",2013-10-14,NaN,2013-10-14,27,http://archiveofourown.org/works/1004531?view_...,"[4.1563544e-06, 0.0024462685, 4.1563544e-06, 4...",0.189626


In [36]:
tfidf_df = pd.read_csv(tfidf_path + 'harry_potter_tfidf_merged_chapters_20230115_largest_rel.tsv', sep='\t')

In [37]:
tfidf_df.head(2)

,index,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,...,Rating,Relationship,Title,Words,PublishDate,UpdateDate,CompleteDate,Comments,URL,Cos
0,798,"BDSM, Bloodplay, Knifeplay, Breathplay, Violen...",Major Character Death,literaryspell,0,M/M,1,"Harry Potter, Draco Malfoy",['Harry Potter - J. K. Rowling'],1013.0,...,Explicit,Draco Malfoy/Harry Potter,Mercy,2377.0,2009-12-28,NaN,2009-12-28,4,http://archiveofourown.org/works/246326?view_a...,0.203293
1,818,"Background Cannibalism, Character Death, Slash...",No Archive Warnings Apply,Catsintheattic,5,M/M,1,"Draco Malfoy, Harry Potter, Original Character...",['Harry Potter - J. K. Rowling'],540.0,...,Teen And Up Audiences,Draco Malfoy/Harry Potter,The Way To a Man\xe2\x80\x99s Heart (How Malfo...,23548.0,2009-12-05,NaN,2009-12-05,0,http://archiveofourown.org/works/712532,0.218885


In [38]:
del tfidf_df['index']
del lda_df['index']

In [39]:
lda_df['UpdateDate'] = lda_df['UpdateDate'].astype('object')
df = pd.merge(tfidf_df, lda_df, on=['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks', 'Category',\
'Characters','Comments' ,'CompleteDate',\
'Fandoms', 'Hits' ,'Kudos', 'Language' ,'PublishDate' ,'Rating',\
'Relationship','Title'  ,'UpdateDate' ,'Words', 'URL', 'Chapters'], how='inner')

In [40]:
df.head(2)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,PublishDate,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD
0,"Age Regression/De-Aging, Kidnapping, Memory Lo...",Creator Chose Not To Use Archive Warnings,Ischa,49,M/M,1,"Pansy Parkinson, Blaise Zabini",['Harry Potter - J. K. Rowling'],6132.0,355.0,...,2013-10-03,NaN,2013-10-03,36,http://archiveofourown.org/works/989371?view_a...,0.219313,0,\n\n~One~\n\n\xe2\x80\x9cYou\xe2\x80\x99ve got...,"[3.5611445e-06, 1.0047966e-05, 3.5611445e-06, ...",0.280998
1,"Angst, Dementors (Harry Potter) - Freeform, Dark",Creator Chose Not To Use Archive Warnings,Frayach,7,M/M,1,"Harry Potter, Draco Malfoy, Hermione Granger",['Harry Potter - J. K. Rowling'],1801.0,64.0,...,2013-10-14,NaN,2013-10-14,27,http://archiveofourown.org/works/1004531?view_...,0.242240,1,"\nNot just a kiss, but The Kiss. The kind of k...","[4.1563544e-06, 0.0024462685, 4.1563544e-06, 4...",0.189626


In [41]:
len(df)

120

In [42]:
topic_words = pickle.load(open('../../data/'+'harry_potter_lda_sample_topic_words.p', 'rb'))

In [43]:
for item in topic_words:
    print(item)
    print()

(0, ['said', 'malfoy', 'her', 'potter', 'we', 'she', 'my', 'your', 'get', 'other', 'well', 'did', 'going', 'himself', 'want', 'away', 'think', 'who', 'looked', 'right'])

(1, ['said', 'your', 'she', 'her', 'malfoy', 'my', 'himself', 'harrys', 'can', 'its', 'see', 'looked', 'something', 'really', 'we', 'did', 'going', 'potter', 'well', 'dracos'])

(2, ['malfoy', 'potter', 'her', 'said', 'she', 'your', 'we', 'can', 'my', 'who', 'off', 'asked', 'something', 'thought', 'himself', 'than', 'away', 'didnt', 'did', 'its'])

(3, ['said', 'potter', 'her', 'she', 'my', 'did', 'can', 'dracos', 'malfoy', 'who', 'ron', 'going', 'himself', 'right', 'your', 'something', 'think', 'its', 'room', 'away'])

(4, ['her', 'malfoy', 'she', 'said', 'potter', 'my', 'we', 'your', 'see', 'thought', 'who', 'than', 'off', 'away', 'himself', 'asked', 'harrys', 'well', 'other', 'its'])

(5, ['said', 'potter', 'malfoy', 'my', 'her', 'she', 'harrys', 'ron', 'himself', 'something', 'think', 'off', 'go', 'your', 'dracos'

In [44]:
df['term_minus_topic'] = df['Cos'] - df['JSD']

In [45]:
df.sort_values(by=['term_minus_topic'], ascending=False).head(2)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
15,"Drabble Collection, Drabble Sequence, literall...",No Archive Warnings Apply,QueenyMidas,11,GenM/MOther,8,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1736.0,99.0,...,2015-04-22,NaN,13,http://archiveofourown.org/works/1002645/chapt...,0.323307,15,\xc2\xa0 \xc2\xa0Draco\'s foot tapped impatien...,"[2.1782432e-06, 0.033830352, 2.1782432e-06, 2....",0.068346,0.254961
93,Suicide Attempt,Creator Chose Not To Use Archive Warnings,leontina,18,M/M,1,"Draco Malfoy, Harry Potter, Teddy Lupin",['Harry Potter - J. K. Rowling'],2636.0,131.0,...,NaN,2013-10-11,3,http://archiveofourown.org/works/1000044,0.294867,93,"\n\nX\nThe 2nd of May, 1998. Voldemort had die...","[1.3447606e-06, 0.015369848, 1.3447606e-06, 1....",0.118180,0.176686


# term - topic: top 10% and bottom 10%

In [46]:
top_10p = np.percentile(df['term_minus_topic'], 90)
df_top = df[df['term_minus_topic'] > top_10p]

In [47]:
df_top.sort_values(by=['term_minus_topic'], ascending=False).head(2)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
15,"Drabble Collection, Drabble Sequence, literall...",No Archive Warnings Apply,QueenyMidas,11,GenM/MOther,8,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1736.0,99.0,...,2015-04-22,NaN,13,http://archiveofourown.org/works/1002645/chapt...,0.323307,15,\xc2\xa0 \xc2\xa0Draco\'s foot tapped impatien...,"[2.1782432e-06, 0.033830352, 2.1782432e-06, 2....",0.068346,0.254961
93,Suicide Attempt,Creator Chose Not To Use Archive Warnings,leontina,18,M/M,1,"Draco Malfoy, Harry Potter, Teddy Lupin",['Harry Potter - J. K. Rowling'],2636.0,131.0,...,NaN,2013-10-11,3,http://archiveofourown.org/works/1000044,0.294867,93,"\n\nX\nThe 2nd of May, 1998. Voldemort had die...","[1.3447606e-06, 0.015369848, 1.3447606e-06, 1....",0.118180,0.176686


In [48]:
bot_10p = np.percentile(df['term_minus_topic'], 10)
df_bot = df[df['term_minus_topic'] < bot_10p]

In [49]:
df_bot.sort_values(by=['term_minus_topic'], ascending=False).head(2)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
43,"Halloween, Costume Parties & Masquerades",No Archive Warnings Apply,disapparater,1,M/M,1,"Draco Malfoy, Harry Potter, Pansy Parkinson, L...",['Harry Potter - J. K. Rowling'],750.0,20.0,...,NaN,2013-10-28,4,http://archiveofourown.org/works/1022160,0.157766,43,\nHarry was taking far too long and Draco was ...,"[1.2076383e-05, 0.0003532935, 1.2076383e-05, 1...",0.435394,-0.277628
40,Halloween,No Archive Warnings Apply,disapparater,5,M/M,1,"Draco Malfoy, Harry Potter, Pansy Parkinson",['Harry Potter - J. K. Rowling'],946.0,31.0,...,NaN,2013-10-21,8,http://archiveofourown.org/works/1012884,0.298112,40,"\n""This is tradition, Harry.\xe2\x80\x9d\n\xe2...","[2.410774e-05, 2.410774e-05, 2.410774e-05, 2.4...",0.578036,-0.279924


In [50]:
df.columns.values

array(['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks',
       'Category', 'Chapters', 'Characters', 'Fandoms', 'Hits', 'Kudos',
       'Language', 'Rating', 'Relationship', 'Title', 'Words',
       'PublishDate', 'UpdateDate', 'CompleteDate', 'Comments', 'URL',
       'Cos', 'Unnamed: 0', 'Text', 'Dist', 'JSD', 'term_minus_topic'],
      dtype=object)

In [52]:
len(df_b)

12

In [53]:
df_top

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
2,"Angst, Flashbacks, Letters",No Archive Warnings Apply,BelladonnaLee,1,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],350.0,3.0,...,NaN,2013-10-22,0,http://archiveofourown.org/works/1012804,0.261566,2,\nIt was a grey morning; snow was falling like...,"[4.7226517e-06, 0.0008700652, 4.7226517e-06, 4...",0.130381,0.131185
13,"Cross Over, Harry potter characters inserted i...",No Archive Warnings Apply,PitchGold,12,M/M,1,"Harry Potter, Draco Malfoy","['Harry Potter - J. K. Rowling', 'Supernatural']",2006.0,82.0,...,NaN,2013-10-11,1,http://archiveofourown.org/works/1000154?view_...,0.438574,13,\nThe night air all around Harry was thick. A ...,"[1.6071538e-06, 1.6071538e-06, 1.6071538e-06, ...",0.272153,0.166421
15,"Drabble Collection, Drabble Sequence, literall...",No Archive Warnings Apply,QueenyMidas,11,GenM/MOther,8,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1736.0,99.0,...,2015-04-22,NaN,13,http://archiveofourown.org/works/1002645/chapt...,0.323307,15,\xc2\xa0 \xc2\xa0Draco\'s foot tapped impatien...,"[2.1782432e-06, 0.033830352, 2.1782432e-06, 2....",0.068346,0.254961
19,"Fae & Fairies, Inheritance, Romance, Fluff",Creator Chose Not To Use Archive Warnings,sky_blue44,4,M/M,1,"Harry Potter, Draco Malfoy, Pansy Parkinson, H...",['Harry Potter - J. K. Rowling'],1539.0,48.0,...,NaN,2013-10-27,1,http://archiveofourown.org/works/1020662,0.283314,19,\nDraco put his spoon down on his plate and lo...,"[7.15788e-06, 0.12988739, 7.15788e-06, 7.15788...",0.142795,0.140519
21,"First Time, Alternate Universe, Hand Jobs, Blo...",No Archive Warnings Apply,Maeglin_Yedi,17,M/M,1,"Harry Potter, Draco Malfoy, Ron Weasley, Hermi...",['Harry Potter - J. K. Rowling'],2158.0,92.0,...,NaN,2013-10-17,9,http://archiveofourown.org/works/1007171?view_...,0.219556,21,"\n""Five points from Gryffindor. Now stop stari...","[0.0004788842, 0.0068050763, 9.0450806e-07, 4....",0.083815,0.135742
34,HP: EWE,No Archive Warnings Apply,Femme,206,M/M,1,"Draco Malfoy, Harry Potter, Luna Lovegood",['Harry Potter - J. K. Rowling'],20462.0,1256.0,...,NaN,2013-10-29,20,http://archiveofourown.org/works/1023084?view_...,0.235330,34,\n\n\n\nCover Design by Tari Sue\n\n\ni.\nHarr...,"[1.3081374e-06, 0.0077986955, 1.3081374e-06, 1...",0.096296,0.139033
39,"HP: EWE, Minor Character Death, Original Chara...",No Archive Warnings Apply,Nherizu,57,M/M,2,"Draco Malfoy, Harry Potter, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],7611.0,232.0,...,NaN,2013-10-28,46,http://archiveofourown.org/works/1017683/chapt...,0.211751,39,\nPART 1\nCover Design by Winter_June\n\nOne\n...,"[0.00033913733, 0.011966574, 3.9300758e-07, 0....",0.044729,0.167023
74,"Romantic Comedy, Drunk Sex, Frenemies, Blow Jo...",No Archive Warnings Apply,Maeglin_Yedi,57,M/M,1,"Harry Potter, Draco Malfoy, Ron Weasley, Hermi...",['Harry Potter - J. K. Rowling'],5311.0,331.0,...,NaN,2013-10-17,21,http://archiveofourown.org/works/1007699?view_...,0.225185,74,\n\nHarry\'s first drink: \n\nThe Ministry ser...,"[1.716086e-06, 0.0010185267, 1.716086e-06, 1.7...",0.091054,0.134132
93,Suicide Attempt,Creator Chose Not To Use Archive Warnings,leontina,18,M/M,1,"Draco Malfoy, Harry Potter, Teddy Lupin",['Harry Potter - J. K. Rowling'],2636.0,131.0,...,NaN,2013-10-11,3,http://archiveofourown.org/works/1000044,0.294867,93,"\n\nX\nThe 2nd of May, 1998. Voldemort had die...","[1.3447606e-06, 0.015369848, 1.3447606e-06, 1....",0.118180,0.176686
98,"Torture, Romance, Angst, Friendship, Captivity","Creator Chose Not To Use Archive Warnings, Gra...",SunseticMonster,35,M/M,3,"Harry Potter, Draco Malfoy, Luna Lovegood",['Harry Potter - J. K. Rowling'],2055.0,162.0,...,NaN,2013-10-11,3,http://archiveofourown.org/works/1000329/chapt...,0.200636,98,The smoky valleys of the Scottish countryside ...,"[3.899214e-05

In [56]:
df_top.AdditionalTags.tolist()

['Angst, Flashbacks, Letters',
 'Cross Over, Harry potter characters inserted into Supernatural world, Alternate Universe, Dark, Draco is a Demon, Harry is a Hunter, Explicit Sexual Content',
 'Drabble Collection, Drabble Sequence, literally every au under the sun',
 'Fae & Fairies, Inheritance, Romance, Fluff',
 'First Time, Alternate Universe, Hand Jobs, Blow Jobs, Anal Sex, Humor, Romantic Comedy',
 'HP: EWE',
 'HP: EWE, Minor Character Death, Original Character(s)',
 'Romantic Comedy, Drunk Sex, Frenemies, Blow Jobs, Anal Sex, Sexual Humor',
 'Suicide Attempt',
 'Torture, Romance, Angst, Friendship, Captivity',
 nan,
 nan]

In [55]:
df_top.head(1).Text.values

array(['\\nIt was a grey morning; snow was falling like ashes.  It was a scenery of peace and tranquillity, but all would be shattered when the first drop of blood spills on to the lily white earth -- the final battle was about to begin.In one of the many rooms in number twelve, Grimmauld Place, Harry Potter was performing his final check.  His wand was in working condition, and the pocket knife he always carried with him was stowed away safely in his pocket.  There was just one more thing left to do.Cloudy green eyes gazed intently at a plain wooden box laid open on the scarred cherry wood desk.  Inside the box were a stack of fading parchments bound by a scarlet-and-gold tie, a bouquet of withered white gardenia, and a smaller black box that could fit neatly in Harry\\\'s palm.With hard eyes he stared at the small box for what seemed like an eternity, before he picked it up with shaky fingers.  After so many years, his heartbeats still raced every time he took it out, as though it wa

In [58]:
df_bot

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
11,Community: hogwarts365,No Archive Warnings Apply,digthewriter,7,M/M,3,NaN,['Harry Potter - J. K. Rowling'],1817.0,97.0,...,NaN,2015-08-27,10,http://archiveofourown.org/works/1024730/chapt...,0.269005,11,Harry was an hour late to Ron\xe2\x80\x99s sur...,"[3.9167127e-05, 3.9167127e-05, 3.9167127e-05, ...",0.610063,-0.341058
12,"Costume Parties & Masquerades, Nature, Halloween",Creator Chose Not To Use Archive Warnings,Ischa,0,M/M,1,NaN,['Harry Potter - J. K. Rowling'],684.0,23.0,...,NaN,2013-10-29,2,http://archiveofourown.org/works/1023217,0.172111,12,\nHarry was lost in the maze. He knew it and h...,"[1.9555757e-05, 1.9555757e-05, 1.9555757e-05, ...",0.515101,-0.342990
31,Gender Issues,Creator Chose Not To Use Archive Warnings,lucifersshroud,6,GenOther,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],672.0,38.0,...,NaN,2013-10-09,2,http://archiveofourown.org/works/998122,0.235814,31,\nIt was the first time Draco had tried puttin...,"[3.383319e-05, 3.383319e-05, 3.383319e-05, 3.3...",0.543771,-0.307958
40,Halloween,No Archive Warnings Apply,disapparater,5,M/M,1,"Draco Malfoy, Harry Potter, Pansy Parkinson",['Harry Potter - J. K. Rowling'],946.0,31.0,...,NaN,2013-10-21,8,http://archiveofourown.org/works/1012884,0.298112,40,"\n""This is tradition, Harry.\xe2\x80\x9d\n\xe2...","[2.410774e-05, 2.410774e-05, 2.410774e-05, 2.4...",0.578036,-0.279924
42,"Halloween, Community: hd_writers, Tricks for T...",Creator Chose Not To Use Archive Warnings,cyn_ful,2,M/M,4,"Harry Potter, Draco Malfoy",['Harry Potter - J. K. Rowling'],99.0,3.0,...,2013-10-24,NaN,2,http://archiveofourown.org/works/1016728/chapt...,0.220552,42,The first Halloween after the war was a diffic...,"[3.3154025e-05, 3.3154025e-05, 3.3154025e-05, ...",0.614433,-0.393882
43,"Halloween, Costume Parties & Masquerades",No Archive Warnings Apply,disapparater,1,M/M,1,"Draco Malfoy, Harry Potter, Pansy Parkinson, L...",['Harry Potter - J. K. Rowling'],750.0,20.0,...,NaN,2013-10-28,4,http://archiveofourown.org/works/1022160,0.157766,43,\nHarry was taking far too long and Draco was ...,"[1.2076383e-05, 0.0003532935, 1.2076383e-05, 1...",0.435394,-0.277628
53,"Ice Cream, Walking, Fluff",Creator Chose Not To Use Archive Warnings,firethesound,10,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],3360.0,320.0,...,NaN,2013-10-24,23,http://archiveofourown.org/works/2240304,0.333560,53,"\nSomewhere along the line, Saturday afternoon...","[1.6062846e-05, 1.6062846e-05, 1.6062846e-05, ...",0.635604,-0.302044
64,"Oral Sex, Mild Language",No Archive Warnings Apply,NathalieWeasley,2,M/M,1,"Harry Potter, Draco Malfoy",['Harry Potter - J. K. Rowling'],323.0,19.0,...,NaN,2013-10-12,0,http://archiveofourown.org/works/1001331?view_...,0.221972,64,\nDraco slips into the club past the over-buil...,"[1.6854316e-05, 0.004640512, 1.6854316e-05, 1....",0.607134,-0.385163
87,"Slash, Muggle/Wizard Relations, Muggles, Rain",No Archive Warnings Apply,digthewriter,13,M/M,1,NaN,['Harry Potter - J. K. Rowling'],1823.0,81.0,...,NaN,2013-10-23,3,http://archiveofourown.org/works/1014625?view_...,0.209781,87,\nThis was the third time that Harry was tryin...,"[7.3409824e-06, 7.3409824e-06, 7.3409824e-06, ...",0.525845,-0.316064
91,"Submission, Bondage, Spanking, Whipping, Anal ...",Creator Chose Not To Use Archive Warnings,HPFangirl71,20,M/M,1,"Harry Potter, Draco Malfoy",['Harry Potter - J. K. Rowling'],7866.0,173.0,...,NaN,2013-10-06,8,http://archiveofourown.org/works/993442?view_a...,0.327850,91,"\nHarry tumbled out of the Irish pub, face flu...","[1.5949307e-05, 1.5949307e-05, 1.5949307e-05, ...",0.634323,-0.306473


# four quardrants

In [77]:
term_top_30p = np.percentile(df['Cos'], 70)
term_bot_30p = np.percentile(df['Cos'], 30)

In [82]:
high_term_high_topic = df[df['Cos'] > term_top_30p]
high_term_high_topic = high_term_high_topic.sort_values(by='JSD', ascending=False).head(12)

In [87]:
high_term_low_topic = df[df['Cos'] > term_top_30p]
high_term_low_topic = high_term_low_topic.sort_values(by='JSD', ascending=True).head(12)

In [88]:
high_term_low_topic

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
15,"Drabble Collection, Drabble Sequence, literall...",No Archive Warnings Apply,QueenyMidas,11,GenM/MOther,8,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1736.0,99.0,...,2015-04-22,NaN,13,http://archiveofourown.org/works/1002645/chapt...,0.323307,15,\xc2\xa0 \xc2\xa0Draco\'s foot tapped impatien...,"[2.1782432e-06, 0.033830352, 2.1782432e-06, 2....",0.068346,0.254961
89,"Snow, Romance, Second War with Voldemort",No Archive Warnings Apply,BelladonnaLee,0,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],16542.0,0.0,...,NaN,2013-10-21,0,http://archiveofourown.org/works/1012581,0.306511,89,"\nWhite powders fell from the light red sky, b...","[9.505998e-06, 9.505998e-06, 9.505998e-06, 9.5...",0.228452,0.078059
66,"Romance, Birthday, Birthday Cake, Fairy Tale E...",Creator Chose Not To Use Archive Warnings,BelladonnaLee,3,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],634.0,15.0,...,NaN,2013-10-23,0,http://archiveofourown.org/works/1012853?view_...,0.326133,66,"\nThe clock struck twelve, the hour when the m...","[7.419574e-06, 0.005545515, 7.419574e-06, 7.41...",0.233036,0.093098
110,"what if, Angst, slightly fluffy, Drarry After ...",No Archive Warnings Apply,Lillielle,23,GenM/M,1,"Draco Malfoy, Harry Potter, Lucius Malfoy, Nar...",['Harry Potter - J. K. Rowling'],2459.0,98.0,...,2013-10-02,NaN,0,http://archiveofourown.org/works/988359?view_a...,0.318084,110,"It was chance, more than anything else, that s...","[1.1413839e-05, 1.1413839e-05, 1.1413839e-05, ...",0.247527,0.070557
9,"Character Death, Early Halloween fic, Bit gore...",Major Character Death,SasuNarufan13,4,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1675.0,43.0,...,NaN,2013-10-30,2,http://archiveofourown.org/works/1024847?view_...,0.323442,9,\n\n\n\nDeadly dance\n\nShining grey eyes glea...,"[1.0533439e-05, 1.0533439e-05, 1.0533439e-05, ...",0.267175,0.056267
13,"Cross Over, Harry potter characters inserted i...",No Archive Warnings Apply,PitchGold,12,M/M,1,"Harry Potter, Draco Malfoy","['Harry Potter - J. K. Rowling', 'Supernatural']",2006.0,82.0,...,NaN,2013-10-11,1,http://archiveofourown.org/works/1000154?view_...,0.438574,13,\nThe night air all around Harry was thick. A ...,"[1.6071538e-06, 1.6071538e-06, 1.6071538e-06, ...",0.272153,0.166421
102,"Vignette, Unrequited Love, Introspection",No Archive Warnings Apply,BelladonnaLee,0,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],292.0,7.0,...,NaN,2013-10-22,0,http://archiveofourown.org/works/1013412,0.339649,102,\nIs this some kind of a joke? Harry thought t...,"[1.2826646e-05, 1.2826646e-05, 1.2826646e-05, ...",0.274036,0.065613
92,Suicidal Thoughts,No Archive Warnings Apply,Sophie_French,19,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],2108.0,166.0,...,NaN,2013-10-09,26,http://archiveofourown.org/works/997893?view_a...,0.348737,92,\n\n\n\n\nWednesday\n\n\n\nFive days.In five d...,"[7.461006e-06, 7.461006e-06, 7.461006e-06, 7.4...",0.287555,0.061182
94,"Surreal, Red Riding Hood Elements, Wine",No Archive Warnings Apply,BelladonnaLee,0,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],250.0,5.0,...,NaN,2013-10-22,0,http://archiveofourown.org/works/1013398,0.330888,94,\nThe wooden clock on the mantelpiece chimed o...,"[1.1473231e-05, 1.1473231e-05, 1.1473231e-05, ...",0.328350,0.002538
30,"Friendship, Developing Relationship",Creator Chose Not To Use Archive Warnings,firethesound,35,M/M,1,"Draco Malfoy, Harry Potter, Ron Weasley, Hermi...",['Harry Potter - J. K. Rowling'],3813.0,328.0,...,NaN,2013-10-17,21,http://archiveofourown.org/works/1162835?view_...,0.312395,30,\nThe sharp rapping of Hermione\'s knuckles on...,"[1.0316902e-05, 1.0316902e-05, 1.0316902e-05, ...",0.331408,-0.019012


In [89]:
high_term_high_topic

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
115,NaN,Graphic Depictions Of Violence,kisslicknipsuck,2,M/M,1,"Harry Potter, Draco Malfoy",['Harry Potter - J. K. Rowling'],959.0,27.0,...,NaN,2013-10-21,6,http://archiveofourown.org/works/1012927,0.365293,115,\nScreams drew his attention. Harry\'d been pe...,"[2.867955e-05, 2.867955e-05, 2.867955e-05, 2.8...",0.686737,-0.321445
103,"Vignette, vampire",No Archive Warnings Apply,BelladonnaLee,1,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],416.0,16.0,...,NaN,2013-10-28,0,http://archiveofourown.org/works/1024589?view_...,0.447913,103,"\n""All right, I will give you my blood. I will...","[2.5956233e-05, 2.5956233e-05, 2.5956233e-05, ...",0.646639,-0.198726
53,"Ice Cream, Walking, Fluff",Creator Chose Not To Use Archive Warnings,firethesound,10,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],3360.0,320.0,...,NaN,2013-10-24,23,http://archiveofourown.org/works/2240304,0.333560,53,"\nSomewhere along the line, Saturday afternoon...","[1.6062846e-05, 1.6062846e-05, 1.6062846e-05, ...",0.635604,-0.302044
91,"Submission, Bondage, Spanking, Whipping, Anal ...",Creator Chose Not To Use Archive Warnings,HPFangirl71,20,M/M,1,"Harry Potter, Draco Malfoy",['Harry Potter - J. K. Rowling'],7866.0,173.0,...,NaN,2013-10-06,8,http://archiveofourown.org/works/993442?view_a...,0.327850,91,"\nHarry tumbled out of the Irish pub, face flu...","[1.5949307e-05, 1.5949307e-05, 1.5949307e-05, ...",0.634323,-0.306473
61,"No Dialogue, HP: EWE, Post - Deathly Hallows, ...",Major Character Death,Queenie_Mab,4,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1351.0,38.0,...,NaN,2013-10-09,10,http://archiveofourown.org/works/997764,0.379882,61,\nI cover my mouth with my arm as I watch Rowl...,"[1.3618868e-05, 1.3618868e-05, 1.3618868e-05, ...",0.622577,-0.242696
27,"Fluff, Mpreg",Creator Chose Not To Use Archive Warnings,leontina,40,M/M,1,"Harry Potter, Draco Malfoy, Teddy Lupin",['Harry Potter - J. K. Rowling'],6743.0,319.0,...,NaN,2013-10-11,13,http://archiveofourown.org/works/1000034,0.477494,27,"\n\nX\n\nTeddy Lupin was four years old, and h...","[4.380372e-06, 4.998503e-05, 4.380372e-06, 4.3...",0.585642,-0.108148
95,"Temper Tantrums, Slash, Male Slash, Gay Male C...",No Archive Warnings Apply,DeiStarr,3,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],815.0,28.0,...,NaN,2013-10-18,1,http://archiveofourown.org/works/1008971,0.387979,95,\nTitle:\xc2\xa0MorningDisclaimer:\xc2\xa0Jo R...,"[2.281787e-05, 2.281787e-05, 2.281787e-05, 2.2...",0.584967,-0.196988
40,Halloween,No Archive Warnings Apply,disapparater,5,M/M,1,"Draco Malfoy, Harry Potter, Pansy Parkinson",['Harry Potter - J. K. Rowling'],946.0,31.0,...,NaN,2013-10-21,8,http://archiveofourown.org/works/1012884,0.298112,40,"\n""This is tradition, Harry.\xe2\x80\x9d\n\xe2...","[2.410774e-05, 2.410774e-05, 2.410774e-05, 2.4...",0.578036,-0.279924
75,"Same-Sex Marriage, Slash, Male Slash, Children...",No Archive Warnings Apply,DeiStarr,3,M/M,1,"Draco Malfoy, Harry Potter, Original Female Ch...",['Harry Potter - J. K. Rowling'],730.0,32.0,...,NaN,2013-10-18,1,http://archiveofourown.org/works/1008949,0.451706,75,\nTitle:\xc2\xa0My Little WrackspurtsDisclaime...,"[1.5720407e-05, 1.5720407e-05, 1.5720407e-05, ...",0.561353,-0.109646
88,"Snakes, Oneshot, Flash Fic",Creator Chose Not To Use Archive Warnings,Sirenfox,0,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],870.0,24.0,...,NaN,2013-10-13,1,http://archiveofourown.org/works/1002067,0.360106,88,"\n""Potter."" Draco greeted the other wizard wit...","[2.6085743e-05, 2.6085743e-05, 2.6085743e-05, ...",0.552885,-0.192779


In [93]:
low_term_high_topic = df[df['Cos'] < term_bot_30p]
low_term_high_topic = low_term_high_topic.sort_values(by='JSD', ascending=False).head(12)

In [94]:
low_term_low_topic = df[df['Cos'] < term_bot_30p]
low_term_low_topic = low_term_low_topic.sort_values(by='JSD', ascending=True).head(12)

In [95]:
low_term_high_topic

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
12,"Costume Parties & Masquerades, Nature, Halloween",Creator Chose Not To Use Archive Warnings,Ischa,0,M/M,1,NaN,['Harry Potter - J. K. Rowling'],684.0,23.0,...,NaN,2013-10-29,2,http://archiveofourown.org/works/1023217,0.172111,12,\nHarry was lost in the maze. He knew it and h...,"[1.9555757e-05, 1.9555757e-05, 1.9555757e-05, ...",0.515101,-0.342990
43,"Halloween, Costume Parties & Masquerades",No Archive Warnings Apply,disapparater,1,M/M,1,"Draco Malfoy, Harry Potter, Pansy Parkinson, L...",['Harry Potter - J. K. Rowling'],750.0,20.0,...,NaN,2013-10-28,4,http://archiveofourown.org/works/1022160,0.157766,43,\nHarry was taking far too long and Draco was ...,"[1.2076383e-05, 0.0003532935, 1.2076383e-05, 1...",0.435394,-0.277628
57,"Mild Language, Aftermath of Violence",Creator Chose Not To Use Archive Warnings,Sanguied,1,M/M,1,NaN,['Harry Potter - J. K. Rowling'],445.0,11.0,...,NaN,2013-10-13,4,http://archiveofourown.org/works/1003059,0.167029,57,\nHarry knew Malfoy was going to go off. It wa...,"[6.6566727e-06, 6.6566727e-06, 6.6566727e-06, ...",0.420533,-0.253504
52,"I\'m Sorry, Past Character Death, Angst, Harry...",Major Character Death,Kandakicksass,2,M/M,1,"Harry Potter, Abraxas Malfoy (sort of)",['Harry Potter - J. K. Rowling'],494.0,13.0,...,NaN,2013-10-17,1,http://archiveofourown.org/works/1007934,0.141557,52,"\nHarry Potter wanders up a long entryway, thr...","[1.3680687e-05, 1.3680687e-05, 1.3680687e-05, ...",0.345130,-0.203573
71,"Romance, Friendship, Depression",Creator Chose Not To Use Archive Warnings,xenadragon_xoxo,3,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],908.0,27.0,...,NaN,2013-10-30,0,http://archiveofourown.org/works/1024485,0.160345,71,\nThere are nights when the wolves are silent ...,"[1.0127676e-05, 0.01774807, 1.0127676e-05, 1.0...",0.341159,-0.180814
62,"No Dialogue, Implied Sexual Content, Romance, ...",Creator Chose Not To Use Archive Warnings,xenadragon_xoxo,14,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1226.0,72.0,...,NaN,2013-10-10,2,http://archiveofourown.org/works/998312?view_a...,0.171491,62,\n\n\n\nLove is not about how much you say I l...,"[1.1442418e-05, 0.061520215, 1.1442418e-05, 1....",0.318523,-0.147032
65,"Romance, Angst, Halloween, Song Lyrics, Song fic",Creator Chose Not To Use Archive Warnings,xenadragon_xoxo,2,M/M,1,"Harry Potter, Draco Malfoy, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],672.0,17.0,...,NaN,2013-10-27,0,http://archiveofourown.org/works/1019907,0.144021,65,\nThings tended to happen to Harry on Hallowee...,"[1.00033185e-05, 1.00033185e-05, 1.00033185e-0...",0.296508,-0.152487
108,"silliness, five times fic, Boys Kissing",No Archive Warnings Apply,who_la_hoop,81,M/M,1,"Draco Malfoy, Harry Potter, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],6857.0,585.0,...,NaN,2013-10-01,29,http://archiveofourown.org/works/987197?view_a...,0.180911,108,\n\n\n[one]\n\nDraco Malfoy looked at the File...,"[3.4159686e-06, 3.4159686e-06, 3.4159686e-06, ...",0.260250,-0.079339
80,"Slash, Light Bondage",Creator Chose Not To Use Archive Warnings,hazylex62,2,M/M,1,"Draco Malfoy, Harry Potter",['Harry Potter - J. K. Rowling'],1106.0,35.0,...,NaN,2013-10-25,1,http://archiveofourown.org/works/1017469?view_...,0.158243,80,"\nWhere was it? Seriously, Where the fuck was ...","[7.456196e-06, 7.456196e-06, 7.456196e-06, 7.4...",0.256239,-0.097996
79,"Slash, HP: EWE",No Archive Warnings Apply,digthewriter,10,M/M,1,Blaise Zabini,['Harry Potter - J. K. Rowling'],2133.0,106.0,...,NaN,2013-10-22,2,http://archiveofourown.org/works/1013448?view_...,0.138937,79,\nDraco hated Muggle streets and Muggle forms ...,"[4.9714968e-06, 0.0061752265, 4.9714968e-06, 4...",0.252969,-0.114032


In [96]:
low_term_low_topic

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Fandoms,Hits,Kudos,...,UpdateDate,CompleteDate,Comments,URL,Cos,Unnamed: 0,Text,Dist,JSD,term_minus_topic
119,NaN,No Archive Warnings Apply,kayladie,8,M/M,1,"Harry Potter, Draco Malfoy, Pansy Parkinson, N...",['Harry Potter - J. K. Rowling'],1405.0,59.0,...,NaN,2013-10-12,0,http://archiveofourown.org/works/1001337?view_...,0.169785,119,\nHarry wasn\'t sure where the impulse came fr...,"[1.654801e-06, 0.011118288, 1.654801e-06, 1.65...",0.038044,0.131741
86,"Slash, Male Slash, Male Homosexuality, Gay Mal...",No Archive Warnings Apply,DeiStarr,67,M/M,4,"Harry Potter, Draco Malfoy, Minerva McGonagall",['Harry Potter - J. K. Rowling'],7673.0,293.0,...,2014-03-17,NaN,31,http://archiveofourown.org/works/995550/chapte...,0.058047,86,Title:\xc2\xa0What Is RequiredDisclaimer:\xc2\...,"[8.7534863e-07, 0.003446861, 8.7534863e-07, 8....",0.044390,0.013657
63,"Non-Linear Narrative, Minor Character Death",No Archive Warnings Apply,kayladie,9,M/M,9,"Harry Potter, Draco Malfoy, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],1007.0,38.0,...,NaN,2013-10-12,1,http://archiveofourown.org/works/1001234/chapt...,0.174157,63,"\n\n31 May, 2001\n\n\nEvery inch of Harry\'s b...","[0.0014154527, 0.012078061, 1.0159804e-06, 0.0...",0.047660,0.126498
67,"Romance, Bonding, Hurt/Comfort",Creator Chose Not To Use Archive Warnings,xenadragon_xoxo,37,M/M,16,"Harry Potter, Draco Malfoy, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],3190.0,152.0,...,NaN,2013-10-12,19,http://archiveofourown.org/works/990164/chapte...,0.136711,67,Something was missing.\nHarry couldn\'t quite ...,"[0.0005743361, 0.0035321268, 3.7113182e-07, 6....",0.059083,0.077628
38,"HP: EWE, Dementors (Harry Potter) - Freeform, ...",Creator Chose Not To Use Archive Warnings,khasael,83,M/M,1,"Harry Potter, Draco Malfoy, Ron Weasley, Hermi...",['Harry Potter - J. K. Rowling'],10105.0,340.0,...,NaN,2013-10-10,28,http://archiveofourown.org/works/990428,0.176891,38,\n\n\n\nCover Design by evening12\nHarry loved...,"[0.000828289, 0.003848375, 4.892626e-07, 2.592...",0.059509,0.117382
59,Mpreg,No Archive Warnings Apply,kayladie,54,M/M,4,"Harry Potter, Draco Malfoy, Pansy Parkinson, R...",['Harry Potter - J. K. Rowling'],6844.0,343.0,...,NaN,2013-10-12,8,http://archiveofourown.org/works/1001183/chapt...,0.171025,59,Lucius Malfoy was a very ordered man. He like...,"[9.4524126e-05, 0.018595371, 1.3530837e-06, 1....",0.060121,0.110904
118,NaN,No Archive Warnings Apply,kayladie,5,M/M,3,"Draco Malfoy, Harry Potter, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],1234.0,55.0,...,NaN,2013-10-12,1,http://archiveofourown.org/works/1001139/chapt...,0.060553,118,Draco Malfoy was a drunk.\nIt was quite the sh...,"[1.5528918e-06, 0.0072261747, 1.5528918e-06, 0...",0.067775,-0.007221
33,HP: EWE,No Archive Warnings Apply,Emeraldawn,12,M/M,1,NaN,['Harry Potter - J. K. Rowling'],2461.0,69.0,...,NaN,2013-10-02,6,http://archiveofourown.org/works/984121,0.149343,33,\n\n\n\n\n\n\nCover design by Capitu\n\n\n\n\n...,"[1.4883125e-06, 0.007764428, 1.4883125e-06, 8....",0.069817,0.079525
24,"Fluff, Animagus, Squee, Romance, Humour",No Archive Warnings Apply,Phoenixstrike,116,M/M,7,NaN,['Harry Potter - J. K. Rowling'],8444.0,569.0,...,NaN,2013-12-24,26,http://archiveofourown.org/works/1007650/chapt...,0.116832,24,Harry pushed open the glass door of the Stubbi...,"[5.8512956e-07, 0.08256585, 5.8512956e-07, 0.0...",0.073333,0.043500
8,"Cat!Harry, Past Child Abuse, deaged!Harry, Pot...",Creator Chose Not To Use Archive Warnings,Milkamoo97,108,M/M,4,"Harry Potter, Draco Malfoy, Hermione Granger, ...",['Harry Potter - J. K. Rowling'],5766.0,311.0,...,2015-02-28,NaN,56,http://archiveofourown.org/works/988707/chapte...,0.133855,8,"\n\xc2\xa0\n\n""When a good man is hurt, all wh...","[1.525352e-06, 0.0058559207, 1.5228063e-06, 1....",0.079055,0.054800


# word log odds

In [112]:
from math import log,sqrt

def logodds(corpora_dic, bg_counter):
    """ It calculates the log odds ratio of term i's frequency between 
    a target corpus and another corpus, with the prior information from
    a background corpus. Inputs are:
    
    - a dictionary of Counter objects (corpora of our interest)
    - a Counter objects (background corpus)
    
    Output is a dictionary of dictionaries. Each dictionary contains the log 
    odds ratio of each word. 
    
    """
    corp_size = dict([(c, sum(corpora_dic[c].values())) for c in corpora_dic])
    bg_size = sum(bg_counter.values())
    result = dict([(c, {}) for c in corpora_dic])
    
    for name, c in corpora_dic.items():
        for word in c:
            #if 10 > sum(1 for corpus in corpora_dic.values() if corpus[word]):
            #    continue
            
            fi = c[word]
            fj = sum(co.get(word, 0.01) for x, co in corpora_dic.items() if x != name)
            fbg = bg_counter.get(word, 0.01)
            ni = corp_size[name]
            nj = sum(x for idx, x in corp_size.items() if idx != name)
            nbg = bg_size
            oddsratio = log(fi+fbg) - log(ni+nbg-(fi+fbg)) -\
                        log(fj+fbg) + log(nj+nbg-(fj+fbg))
            std = 1.0 / (fi+fbg) + 1.0 / (fj+fbg)
            z = oddsratio / sqrt(std)
            result[name][word] = z
            
    return result

In [113]:
def overrep(df1, df2, df_bg, field):
    ch_list1 = []
    for line in df1[field].tolist():
        ch_list1.extend([item.lower().replace('\\','') for item in line.split()])
        
    ch_list2 = []
    for line in df2[field].tolist():
        ch_list2.extend([item.lower().replace('\\','') for item in line.split()])
    
    ch_list_bg = []
    for line in df_bg[field].tolist():
        ch_list_bg.extend([item.lower().replace('\\','') for item in line.split()])
        
    
    corp_1 = Counter(ch_list1)
    corp_2 = Counter(ch_list2)
    corp_bg = Counter(ch_list_bg)
    

    corp_1 = {k:v for k, v in corp_1.items() if v > 1 }
    corp_2 = {k:v for k, v in corp_2.items() if v > 1 }
    corp_bg = {k:v for k, v in corp_bg.items() if v > 1 }


    result = logodds({'df1':corp_1, 'df2':corp_2}, corp_bg)
    print('Overrep in df1: ')
    print()
    for k in sorted(result['df1'], key=result['df1'].get, reverse=True)[0:15]:
        print(k,result['df1'][k])
    print()
    
    print('Overrep in df2: ')
    print()
    for k in sorted(result['df2'], key=result['df2'].get, reverse=True)[0:15]:
        print(k,result['df2'][k])
#     return sorted(result['fic'], key=result['fic'].get)[0:20]

In [114]:
overrep(high_term_high_topic, high_term_low_topic, df, 'Text')

Overrep in df1: 

daddy 4.3159425473987545
teddy 3.231921679218854
cousin 3.0193675120110766
healer 1.9816279332281865
baby 1.899200472536729
fleur 1.563835204777663
alicia 1.559398720451223
teddy," 1.5192650115917252
limited 1.204607634435738
victoire 1.177460934083427
ice 1.161864072144456
teddy's 1.0737453114180684
pumpkins 1.0640385371333954
brother 1.0494986233269181
malfoy 1.0475237309283247

Overrep in df2: 

demon 7.7954676246761805
demon's 3.4247887710079024
demons 3.3682240930566363
him.nxc2xa0xc2xa0 1.9511503925269884
ginevra 1.5654175499757768
cake 1.5461560688966418
lily 1.4586420385334626
hood 1.3241322273165128
car 1.2848680926041984
too.nxc2xa0xc2xa0 1.2082858837181876
demon, 1.2082858837181876
yowl 1.2082858837181876
demon. 1.2082858837181876
m.a.p. 1.2082858837181876
the 1.196271843887742


In [115]:
overrep(high_term_high_topic, low_term_high_topic, df, 'Text')

Overrep in df1: 

daddy 4.382395463396541
cousin 2.952646222823898
teddy 2.8235506973953806
baby 1.9078686138509278
healer 1.8830445578994628
cream 1.6911760708255639
ice 1.5677306278737329
alicia 1.5301956318122454
fleur 1.5295649731905063
teddy," 1.4835904807749447
harry's 1.4818508591557138
limited 1.1630935997440268
victoire 1.1553776862824894
draco's 1.1355278690493527
didn't 1.0174082996241987

Overrep in df2: 

. 2.748191931586642
xe2x80x93 2.0220617634145706
harryxe2x80x99s 1.8588091354898464
corset 1.7456992996048697
skirt 1.7313391626984675
hexe2x80x99d 1.7087520445524693
cucumber 1.5609485809929493
condom 1.5609485809929493
wasnxe2x80x99t 1.5388762425823297
dracoxe2x80x99s 1.4650830694473829
didnxe2x80x99t 1.4242186120999598
bears 1.232971775121584
itxe2x80x99s 1.1857074624819408
crows 1.1595919613547738
he 1.1257186307421587


In [117]:
overrep(high_term_high_topic, low_term_low_topic, df, 'Text')

Overrep in df1: 

teddy 6.475748260271494
daddy 5.912111523379659
* 3.8925826357978894
cousin 3.730151333574702
the 3.2993365608630767
malfoy 2.771444308533286
-- 2.613237319476505
cream 2.5422502896810357
ice 2.4571602492306264
my 2.3884520866385985
baby 2.3297610607948083
his 2.2611691748124993
teddy's 2.1315308884130717
fleur 2.1227985801524185
teddy," 2.1011330853377617

Overrep in df2: 

xe2x80x93 6.681780754989002
mr. 4.235418477419214
that 3.667327178396825
mulpepper 3.3145593251946632
hermione 3.143514242708634
- 2.883874970861803
cat 2.862059739423288
didn't 2.8149686396613474
wasn't 2.7884807686866986
been 2.724424155381103
he 2.6560050809507723
to 2.6549874131167397
ron 2.5992529213045548
was 2.57199764933747
said, 2.5707389403567324


In [118]:
overrep(high_term_low_topic, low_term_high_topic, df, 'Text')

Overrep in df1: 

demon 7.9908518085642415
demon's 3.508858684792047
demons 3.4540218610721505
him.nxc2xa0xc2xa0 1.9990843786204553
ginevra 1.626407472442139
cake 1.617190136037855
lily 1.556760423607136
the 1.5528735016367734
harry's 1.5508456332213305
a 1.384457378340338
hood 1.3567064161593236
narcissa 1.2969752295584749
circle 1.2544017079487735
too.nxc2xa0xc2xa0 1.2380251747486146
demon, 1.2380251747486146

Overrep in df2: 

. 3.3606279306734286
xe2x80x93 2.7680522719078446
didnxe2x80x99t 1.8646750939250656
corset 1.8226427786852741
skirt 1.8178676349796277
wasnxe2x80x99t 1.8071129691363392
hexe2x80x99d 1.6698003785083078
cucumber 1.6297744860079522
condom 1.6297744860079522
harryxe2x80x99s 1.5218891380267936
itxe2x80x99s 1.4881149668406826
malfoy 1.3990809571278293
donxe2x80x99t 1.3352690816258397
he 1.3323635623766767
muggle 1.3191584797859264


In [119]:
overrep(high_term_low_topic, low_term_low_topic, df, 'Text')

Overrep in df1: 

demon 10.798233595172203
demon's 4.7168482225689
demons 4.6868383724583085
the 4.59800225243551
* 4.021259128301322
-- 3.409660409931484
his 2.848714467594509
him.nxc2xa0xc2xa0 2.687844061652601
lily 2.5673689537475695
ginevra 2.5027656836867616
eyes 2.415931542721708
like 2.3104407628480352
cake 2.3075829362549998
green 2.2819445688945725
white 2.1322528015122355

Overrep in df2: 

xe2x80x93 7.488459655358479
mr. 4.301612597354698
hermione 3.584554203764045
mulpepper 3.5375166467616515
didn't 3.2074338568477962
to 3.203959755493725
- 3.1929112120838696
said, 3.1563823335455075
he 2.8875918492249437
auror 2.7998830267111297
that 2.731019705322849
wasn't 2.6906810427648913
ron 2.5968631060991987
been 2.581997818089152
pansy 2.568662664521946


In [120]:
overrep(low_term_high_topic, low_term_low_topic, df, 'Text')

Overrep in df1: 

. 7.890946718420321
itxe2x80x99s 3.6134249498846587
teddy 3.513471919536849
malfoyxe2x80x99s 3.449168545477359
hexe2x80x99s 3.4091980939871873
malfoy 3.1506031739080473
the 2.943528389462076
skirt 2.4580042871635697
says, 2.4573465137744672
corset 2.3918699924568974
doesnxe2x80x99t 2.3915577832345933
canxe2x80x99t 2.345399194824162
his 2.3448189834994118
against 2.332997440117064
knows 2.1631424525678904

Overrep in df2: 

xe2x80x93 4.628319881098262
mr. 4.308964128186108
didn't 3.895766202487625
mulpepper 3.436389220729753
wasn't 3.369760952879989
- 3.0527408268081984
cat 3.018143128181931
that 2.9916393939874926
hermione 2.852068821486864
auror 2.8054904915679106
lucius 2.755258518028177
you're 2.683979813557155
was 2.5666101349110733
he'd 2.4704457488407185
been 2.4504452533814596


In [121]:
overrep(df_top, df_bot, df, 'Text')

Overrep in df1: 

. 9.160377614676744
demon 5.971183846134889
luna 4.4561993991780176
raines 4.131759813710963
said 3.8307029603240523
harry 3.2644687560310186
sirius 3.131430752674693
her 3.126345258796164
nharry 2.912022884438671
his 2.792320128491968
scorpius 2.755800776223548
n"i 2.6947632206000156
demon's 2.666182732112285
lily 2.5644095040060892
demons 2.524078973824409

Overrep in df2: 

of 2.8815258457659727
hexe2x80x99d 2.435455145277223
pumpkin 2.192202904391381
pumpkins 2.093276409229699
hexe2x80x99s 2.0071834426383477
potterxe2x80x99s 1.9506737663698335
halloween 1.8626217088325794
dracoxe2x80x99s 1.727466872917928
is 1.6693171082553817
ice 1.5979723813183984
barman 1.5854787843168674
dance 1.546592469869303
film 1.5288464437308829
i 1.5059639859214853
maze 1.4806992036801545
